In [ ]:
import os

import lightning as L
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm

import g_main
import module_model

QCGNN = module_model.QCGNN_IX

shots = 1024

general_config = g_main.json_config.copy()
general_config["batch_size"] = 10
general_config["rnd_seed"] = 0
general_config["max_num_ptcs"] = 2
general_config["num_bin_data"] = 100 # {100} * 10 * 2 * (1 - 0.8) = 400 test data
L.seed_everything(general_config["rnd_seed"])

# Quantum configurations.
qidx = 1
qnn, gl, gr  = 7, 1, 7

# Output directory.
ibmq_dir = os.path.join(general_config["predictions_dir"], "ibmq")
os.makedirs(ibmq_dir, exist_ok=True)

In [ ]:
class IBMQQCGNN(QCGNN):
    """Return additional measurement outputs."""
    
    def forward(self, x):
        pt = x[..., 0]
        num_ptcs = torch.sum((pt > 0).float(), axis=-1, keepdim=True)
        x = torch.flatten(x, start_dim=-2, end_dim=-1)
        x = self.net(x)
        x = torch.unflatten(
            x, dim=-1, sizes=(2**self.num_ir_qubits, self.num_nr_qubits))
        
        # Add this line.
        meas = x.detach() # Measurement outputs.

        x = x.mT
        x = torch.sum(x, dim=-1)

        if self.aggregation == "SUM":
            x = x * num_ptcs
        elif self.aggregation == "MEAN":
            x = x / num_ptcs
        
        return x, meas

class IBMQQuantumRotQCGNN(g_main.QuantumRotQCGNN):
    def __init__(self, num_ir_qubits, num_nr_qubits, num_layers, num_reupload, quantum_config):
        """Change phi from QCGNN to IBMQQCGNN."""
        
        super().__init__(
            num_ir_qubits=num_ir_qubits,
            num_nr_qubits=num_nr_qubits,
            num_layers=num_layers,
            num_reupload=num_reupload,
            quantum_config=quantum_config,
            aggregation="SUM",
        )
        
        # Substitute QCGNN with IBMQQCGNN.
        self.phi = IBMQQCGNN(
            num_ir_qubits=num_ir_qubits,
            num_nr_qubits=num_nr_qubits,
            num_layers=num_layers,
            num_reupload=num_reupload,
            ctrl_enc=self.ctrl_enc,
            shots=shots,
            **quantum_config
        )
    
    def forward(self, x):
        # Additional measurement outputs from IBMQQCGNN.
        x, meas = self.phi(x)

        # Original workflow.
        x = self.mlp(x)

        return x.detach(), meas

In [ ]:
def load_state_dict(model: nn.Module, ckpt_path: str):
    """Load model checkpoints parameters.
    
    Args:
        model : nn.Module
            Model to be load state dict.
        ckpt_path : str
            Path of checkpoints.
    """

    # The keys of ckpt state_dict somehow different than loading keys.
    old_state_dict = torch.load(ckpt_path)["state_dict"]
    new_state_dict = {}
    for old_key in old_state_dict.keys():
        # Containing prefix "model.", no need this prefix.
        new_key = old_key[6:]
        print(f"state_dict key updated: {old_key} ---> {new_key}")
        new_state_dict[new_key] = old_state_dict[old_key]
    model.load_state_dict(new_state_dict)

def prediction(x, data_config, quantum_config):
    # Choose pretrained model with number of particles = 16 -> qidx = log2(16) = 43
    model_suffix = f"qidx4_qnn{qnn}_gl{gl}_gr{gr}"

    # Get checkpoints file.
    data_suffix = f"{data_config['abbrev']}_ptc16_thres0.05-nb500_R0"
    ckpt_key = f"QCGNN_IX_{model_suffix}_SUM-{data_suffix}"
    ckpt_path = g_main.get_ckpt_path(ckpt_key, general_config["rnd_seed"])
    
    # Create model and load checkpoints.
    model = IBMQQuantumRotQCGNN(qidx, qnn, gl, gr, quantum_config)
    load_state_dict(model, ckpt_path)
    model.eval()
    
    return model(x)

In [ ]:
# Recreate a new data configuration list.
data_config_list = [
    # 2-prong v.s. 1-prong.
    g_main.generate_data_config(
        sig="VzToZhToVevebb", bkg="VzToQCD", abbrev="BB-QCD",
        cut_pt=(800, 1000), subjet_radius=0, bin=10,
        num_bin_data=general_config["num_bin_data"],
        max_num_ptcs=general_config["max_num_ptcs"],
        pad_num_ptcs=(2 ** qidx),
        pt_threshold=general_config["pt_threshold"],
        num_ptcs_range=(2 ** qidx, 2 ** qidx),
        print_log=True,
    ),
    
    # 3-prong v.s. 1-prong.
    g_main.generate_data_config(
        sig="VzToTt", bkg="VzToQCD", abbrev="TT-QCD",
        cut_pt=(800, 1000), subjet_radius=0, bin=10,
        num_bin_data=general_config["num_bin_data"],
        max_num_ptcs=general_config["max_num_ptcs"],
        pad_num_ptcs=(2 ** qidx),
        pt_threshold=general_config["pt_threshold"],
        num_ptcs_range=(2 ** qidx, 2 ** qidx),
        print_log=True,
    ),
]

In [ ]:
# Choose an IBMQ backend
ibmq_backend = input("Enter IBMQ backend = ") or "ibmq_qasm_simulator"

# Prediction on IBMQ.
for data_config in data_config_list:
    
    # Create the data module.
    data_module = g_main.generate_datamodule(
        data_config=data_config,
        data_ratio=general_config["data_ratio"],
        batch_size=general_config["batch_size"],
        graph=False
    )

    # Buffers for saving data.
    x, y_true, y_penl, meas_penl, y_ibmq, meas_ibmq = [torch.tensor([]) for _ in range(6)]
    for _x, _y_true in tqdm(data_module.test_dataloader()):
        # Prediction on PennyLane.
        _y_penl, _meas_penl = prediction(
            x=_x,
            data_config=data_config,
            quantum_config={"qdevice": "default.qubit", "diff_method": "best", "qbackend": ""}
        )

        # Prediction on IBMQ.
        _y_ibmq, _meas_ibmq = prediction(
            x=_x,
            data_config=data_config,
            quantum_config={"qdevice": "qiskit.ibmq", "diff_method": "parameter-shift", "qbackend": ibmq_backend}
        )

        # Dataset and true labels.
        x, y_true = torch.cat((x, _x)), torch.cat((y_true, _y_true))
        
        # Save predictions to buffers.
        y_penl, meas_penl = torch.cat((y_penl, _y_penl)), torch.cat((meas_penl, _meas_penl))
        y_ibmq, meas_ibmq = torch.cat((y_ibmq, _y_ibmq)), torch.cat((meas_ibmq, _meas_ibmq))

    # Output dictionary    
    result = {
        "x": x, "y_true": y_true,
        "y_penl": y_penl, "meas_penl": meas_penl,
        "y_ibmq": y_ibmq, "meas_ibmq": meas_ibmq
    }

    # Save result to npy files.
    npy_file = (
        # backend - shots - abbrev
        f"{ibmq_backend}-s{shots}-{data_config['abbrev']}_"

        # rnd_seed - max_num_ptcs
        + f"r{general_config['rnd_seed']}-p{general_config['max_num_ptcs']}_"

        # num_bin_data
        + f"n{general_config['num_bin_data']}_"

        # batch_size - qnn_gl_gr
        + f"b{general_config['batch_size']}-{qnn}_{gl}_{gr}"

        # npy suffix.
        + ".npy"
    )
    npy_path = os.path.join(ibmq_dir, npy_file)
    np.save(npy_path, result, allow_pickle=True)